In [15]:
# IMPORTS
import string
from typing import *

In [16]:
# DEFINE CONSTANTS
BOARD_SIZE: int = 10
BOARD_COLUMNS: tuple[str] = tuple(string.ascii_uppercase[:BOARD_SIZE])
PAWN_MAX_RANGE: int = 1
QUEEN_MAX_RANGE: int = 9

In [17]:
# make a checkers board class
class board:
    
    def __init__(self, size: int, columns: list[str]) -> None:
        self.size: int = size
        self.columns: list[str] = columns
        self.board: list[dict] = self.create_board()
    
    def __repr__(self) -> str:
        return f"{self.board}"
    
    def __str__(self) -> str:
        """
        print the board in a nice way with the owner of each cell 
        with the column on the top of the board and the row 
        on the left side of the board
        """
        board_string: str = ""
        column_names: str = "  "
        rows_names: str = ""
        # add column names to the top of the board
        for column in range(self.size):
            column_names += f" {self.columns[column]} "
        # add each cells rows with the row number to the left side of the board
        for row in range(self.size):
            rows_names += f"\n{row} "
            for cell in self.board:
                if cell["cell_row"] == row:
                    rows_names += f" {cell['cell_owner']} "
        board_string = f"{column_names}{rows_names}"
        return board_string
    
    def create_board(self) -> list[dict]:
        """
        This function will create the board
        """
        board: list = []
        for column in self.columns:
            for row in range(self.size):
                cell: dict = {}
                cell["cell_row"] = row
                cell["cell_col"] = column
                cell["cell_index"] = (row, column)
                cell["cell_color"] = "W" if (self.columns.index(column) + row) % 2 == 0 else "B"
                cell["cell_is_empty"] = True
                cell["cell_owner"] = 0
                board.append(cell)
        return board
    
    def get_cell(self, board: list[dict], cell_index: tuple[int, str]) -> tuple[dict, int]:
        """
        this function will return a tuple with the dict that contain 
        informations about the cell and the index of this dict in the board list.
        For searching the cell in the board list we will use disection algorithm.
        """
        i: int = 0
        j: int = len(board) - 1
        while i <= j:
            if board[i]["cell_index"] == cell_index:
                return board[i], i
            else:
                i+=1
            if board[j]["cell_index"] == cell_index:
                return board[j], j
            else:
                j-=1
        return None, None

init_board = board(BOARD_SIZE, BOARD_COLUMNS)

"""board_created = init_board.create_board()
print(bool(init_board.get_cell(board_created, (10, "K"))))
print(board_created[14]) """

print(init_board)


   A  B  C  D  E  F  G  H  I  J 
0  0  0  0  0  0  0  0  0  0  0 
1  0  0  0  0  0  0  0  0  0  0 
2  0  0  0  0  0  0  0  0  0  0 
3  0  0  0  0  0  0  0  0  0  0 
4  0  0  0  0  0  0  0  0  0  0 
5  0  0  0  0  0  0  0  0  0  0 
6  0  0  0  0  0  0  0  0  0  0 
7  0  0  0  0  0  0  0  0  0  0 
8  0  0  0  0  0  0  0  0  0  0 
9  0  0  0  0  0  0  0  0  0  0 


In [18]:
class Pawns:
    
    def create_player_pawns(self, player_id: int, board: list[dict]) -> list:
        """
        this function will create the pawns for the player with the given id
        """
        player_pawns: list[dict] = []
        for cell in board:
            pawn_informations: dict = {}
            if player_id == 1 and cell["cell_row"] > 5 and cell["cell_color"] == "B":
                #Create pawn with its data and add it to the player 1 pawns list
                pawn_informations["pawn_type"] = "Pawn"
                pawn_informations["pawn_color"] = "W"
                pawn_informations["pawn_status"] = "alive"
                pawn_informations["pawn_owner"] = 1
                pawn_informations["pawn_row"] = cell["cell_row"]
                pawn_informations["pawn_col"] = cell["cell_col"]
                pawn_informations["pawn_pos"] = list(cell["cell_index"])
                player_pawns.append(pawn_informations)
                #Update the cell data
                cell["cell_is_empty"] = False
                cell["cell_owner"] = player_id
            elif player_id == 2 and cell["cell_row"] < 4 and cell["cell_color"] == "B":
                pawn_informations["pawn_color"] = "B"
                pawn_informations["pawn_type"] = "Pawn"
                pawn_informations["pawn_status"] = "alive"
                pawn_informations["pawn_owner"] = 2
                pawn_informations["pawn_row"] = cell["cell_row"]
                pawn_informations["pawn_col"] = cell["cell_col"]
                pawn_informations["pawn_pos"] = list(cell["cell_index"])
                player_pawns.append(pawn_informations)
                cell["cell_is_empty"] = False
                cell["cell_owner"] = player_id
        return player_pawns

    def get_pawn(self, pawns: list[dict], pawn_pos: list[int, str]) -> tuple[dict, int]:
        """
        this function will return the pawn that is in the given position
        """
        i: int = 0
        j: int = len(pawns) - 1
        while i <= j:
            if pawns[i]["pawn_pos"] == pawn_pos:
                return pawns[i], i
            else:
                i+=1
            if pawns[j]["pawn_pos"] == pawn_pos:
                return pawns[j], j
            else:
                j-=1
        return None, None
    
    def is_reachable(self, pawn: tuple[dict, int], init_board) -> list[tuple[dict, int]]:
        """
        this function will return the list of the cells that the pawn can move to
        """
        reachable_cells: list[dict] = []
        columns: list[str] = BOARD_COLUMNS
        pawn_type: str = pawn[0]["pawn_type"]
        pawn_owner:int = pawn[0]["pawn_owner"]
        pawn_row: int = pawn[0]["pawn_row"]
        pawn_col: int = columns.index(pawn[0]["pawn_col"])
        
        for i in range(1, PAWN_MAX_RANGE+1 if pawn_type == "Pawn" else QUEEN_MAX_RANGE+1):
            if pawn_owner == 1:
                reach_dest1: tuple[int, str] = (pawn_row - i, columns[pawn_col + i])
                reach_dest2: tuple[int, str] = (pawn_row - i, columns[pawn_col - i])
                if pawn_type == "Queen":
                    reach_dest3: tuple[int, str] = (pawn_row + i, columns[pawn_col - i])
                    reach_dest4: tuple[int, str] = (pawn_row + i, columns[pawn_col + i])
            else:
                reach_dest1: tuple[int, str] = (pawn_row + i, columns[pawn_col + i])
                reach_dest2: tuple[int, str] = (pawn_row + i, columns[pawn_col - i])
                if pawn_type == "Queen":
                    reach_dest3: tuple[int, str] = (pawn_row - i, columns[pawn_col - i])
                    reach_dest4: tuple[int, str] = (pawn_row - i, columns[pawn_col + i])
        
            if None not in init_board.get_cell(init_board.board, reach_dest1):
                reachable_cells.append(init_board.get_cell(init_board.board, reach_dest1))
            if None not in init_board.get_cell(init_board.board, reach_dest2):
                reachable_cells.append(init_board.get_cell(init_board.board, reach_dest2))
            
            if pawn_type == "Queen":
                if None not in init_board.get_cell(init_board.board, reach_dest3):
                    reachable_cells.append(init_board.get_cell(init_board.board, reach_dest3))
                if None not in init_board.get_cell(init_board.board, reach_dest4):
                    reachable_cells.append(init_board.get_cell(init_board.board, reach_dest4))
        
        return reachable_cells

    def move_pawn(self, pawns:list[dict], pawn: tuple[dict, int], move_to: tuple[int, str], init_board) -> bool:
        """
        this function will move the pawn
        return True if the move is good and False otherwise
        """
        informations_about_the_actual_cell: tuple[dict, int] = init_board.get_cell(init_board.board, tuple(pawn[0]["pawn_pos"]))
        informations_about_the_destination_cell: tuple[dict, int] = init_board.get_cell(init_board.board, move_to)
        if pawn[0]["pawn_pos"] != list(move_to):
            if informations_about_the_destination_cell in self.is_reachable(1, pawn, init_board) and informations_about_the_destination_cell[0]["cell_is_empty"]:
                informations_about_the_actual_cell[0]["cell_is_empty"] = True
                informations_about_the_actual_cell[0]["cell_owner"] = 0
                
                pawns[pawn[1]]["pawn_row"] = move_to[0]
                pawns[pawn[1]]["pawn_col"] = move_to[1]
                pawns[pawn[1]]["pawn_pos"] = [pawns[pawn[1]]["pawn_row"], pawns[pawn[1]]["pawn_col"]]
                
                informations_about_the_destination_cell[0]["cell_is_empty"] = False
                informations_about_the_destination_cell[0]["cell_owner"] = pawns[pawn[1]]["pawn_owner"]
                return True
            else:
                return False
        else:
            return False
                

player1: list[dict] = Pawns().create_player_pawns(1, init_board.board)
player2: list[dict] = Pawns().create_player_pawns(2, init_board.board)


#print("Player1 pawns :\n", player1,"\n\n\nPlayer2 pawns :\n", player2,"\n\n\n")

#print(init_board,"\n\n\n")

get_a_pawn_informations: tuple[dict, int] = Pawns().get_pawn(player1, [6, "B"])

print(Pawns().is_reachable(get_a_pawn_informations, init_board), "\n\n\n")

print(Pawns().move_pawn(player1, get_a_pawn_informations, (5, "C"), init_board), "\n\n\n")

print(init_board,"\n\n\n")

[({'cell_row': 5, 'cell_col': 'C', 'cell_index': (5, 'C'), 'cell_color': 'B', 'cell_is_empty': True, 'cell_owner': 0}, 25), ({'cell_row': 5, 'cell_col': 'A', 'cell_index': (5, 'A'), 'cell_color': 'B', 'cell_is_empty': True, 'cell_owner': 0}, 5)] 





TypeError: Pawns.is_reachable() takes 3 positional arguments but 4 were given